# Running normal CPU methods on multiple threads

In [2]:
CODE_ROOT = pwd() * "/"

"/home/rrs/Desktop/MPAS_Ocean/"

In [3]:
include(CODE_ROOT * "mode_init/mode_init.jl")

include(CODE_ROOT * "visualization.jl")

include(CODE_ROOT * "mode_forward/time_steppers.jl")

forward_euler_step_cuda! (generic function with 1 method)

In [4]:
using PyPlot, PyCall
animation  = pyimport("matplotlib.animation")
ipydisplay = pyimport("IPython.display")

PyObject <module 'IPython.display' from '/home/rrs/anaconda3/envs/MPAS_Ocean/lib/python3.8/site-packages/IPython/display.py'>

# Load Mesh From File

In [9]:
mpasOcean = MPAS_Ocean(false, CODE_ROOT * "MPAS_O_Shallow_Water/Mesh+Initial_Condition+Registry_Files/Periodic",
                "base_mesh.nc", "mesh.nc", periodicity="Periodic"
)
typeof(mpasOcean)

MPAS_Ocean

In [10]:
function calculate_normal_velocity_tendency_threaded!(mpasOcean::MPAS_Ocean)
    mpasOcean.normalVelocityTendency[:] .= 0

    Threads.@threads for iEdge in 1:mpasOcean.nEdges
        # gravity term: take gradient of sshCurrent across edge
        cell1Index, cell2Index = mpasOcean.cellsOnEdge[:,iEdge]
        
        if cell1Index !== 0 && cell2Index !== 0
            mpasOcean.normalVelocityTendency[iEdge] = mpasOcean.gravity * ( mpasOcean.sshCurrent[cell1Index] - mpasOcean.sshCurrent[cell2Index] ) / mpasOcean.dcEdge[iEdge]
        end
        
        # coriolis term
        for i in 1:mpasOcean.nEdgesOnEdge[iEdge]
            eoe = mpasOcean.edgesOnEdge[i,iEdge]
            
            mpasOcean.normalVelocityTendency[iEdge] += mpasOcean.weightsOnEdge[i,iEdge] * mpasOcean.normalVelocityCurrent[eoe] * mpasOcean.fEdge[eoe]
        end
    end
end

calculate_normal_velocity_tendency_threaded! (generic function with 1 method)

In [7]:
Threads.nthreads()

12

In [8]:
using BenchmarkTools

In [13]:
@benchmark calculate_normal_velocity_tendency_threaded!(mpasOcean)

BechmarkTools.Trial: 621 samples with 1 evaluations.
 Range (min … max):  2.737 ms … 188.844 ms  ┊ GC (min … max):  0.00% … 84.28%
 Time  (median):     3.684 ms               ┊ GC (median):     0.00%
 Time  (mean ± σ):   8.043 ms ±  23.211 ms  ┊ GC (mean ± σ):  49.91% ± 16.91%

  █▂                                                           
  ██▄▁▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▄▁▄▁▄▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▄▄▁▁▁▅ ▆
  2.74 ms      Histogram: log(frequency) by time       149 ms <

 Memory estimate: 17.50 MiB, allocs estimate: 1011392.

In [14]:
@benchmark calculate_normal_velocity_tendency!(mpasOcean)

BechmarkTools.Trial: 324 samples with 1 evaluations.
 Range (min … max):  14.407 ms … 18.449 ms  ┊ GC (min … max): 0.00% … 17.10%
 Time  (median):     14.474 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   15.452 ms ±  1.478 ms  ┊ GC (mean ± σ):  6.22% ±  8.27%

  ██                                                     ▅▄▁▁  
  ██▇▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▁▁▄▁▁▁▁▁▁▁▁▁▁▆████ ▆
  14.4 ms      Histogram: log(frequency) by time      17.8 ms <

 Memory estimate: 13.24 MiB, allocs estimate: 725293.